# Contour extraction for ground truth masks of DAVIS 2016

In [0]:
!pip install -U -q PyDrive

     |████████████████████████████████| 993kB 2.8MB/s 


In [0]:
# PyDrive (https://colab.research.google.com/notebooks/io.ipynb#scrollTo=7taylj9wpsA2)
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import cv2

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np

import pandas as pd

import re

from scipy import spatial

from skimage import measure

# %matplotlib inline

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Functions

### Drive helpers

In [0]:
def create_folder_in_drive(parent_folder_id, folder_title):
    '''Returns the id of the newly created folder in Drive'''
    
    print('\tCreate folder')
    
    folder_metadata = {'title': folder_title,
                       'mimeType': 'application/vnd.google-apps.folder',
                       'parents': [{u'id' : parent_folder_id}]}
    folder = drive.CreateFile(folder_metadata)
    folder.Upload()
    
    return folder['id']

In [0]:
def download_file_from_drive(file_id, file_name):
    '''Downloads a file from Drive so that it is available in Colab.'''
    
    downloaded = drive.CreateFile({'id': file_id})
    downloaded.GetContentFile(file_name)

In [0]:
def download_images_from_folder(folder_id):
    '''Returns list of names of downloaded images from Drive folder.'''    
    
    print('\tDownloading images')
    file_list = drive.ListFile({'q': ("'{}' in parents and trashed=false").
                                    format(folder_id)}).GetList()
    downloaded_images = []    
    for file in file_list:
        file_title = file['title']
        file_id = file['id']
        
        download_file_from_drive(file_id, file_title)
        
        downloaded_images.append(file_title)
        
    return downloaded_images
    

In [0]:
def upload_file_to_drive(folder_id, file_name):
    '''Uploads file from Colab to Drive folder.'''
    
    file = drive.CreateFile({'parents':[{u'id': folder_id}]})
    file.SetContentFile(file_name)
    file.Upload()

### Save functions

In [0]:
def save_folder_list_as_csv(annotations_folder_id, file_name):
    '''Saves Drive folder list with title and id as csv.'''
    
    folder_list = drive.ListFile({'q': ("'{}' in parents and trashed=false").
                                  format(ANNOTATIONS_480P_FOLDER_ID)}).GetList()
    df = pd.DataFrame.from_dict(folder_list)
    df = df[['title', 'id']]
    df.to_csv(file_name)

In [0]:
def save_image_with_contour(image, contour, file_name):
    '''Saves plotted image with contour.'''
    
    fig, ax = plt.subplots(figsize=(15, 10))
    ax.scatter(contour[:, 0], contour[:, 1])
    ax.imshow(image)
    ax.axis('image')
    ax.set_xticks([])
    ax.set_yticks([])
    plt.savefig(file_name, dpi=300, bbox_inches='tight')    
    plt.close(fig)

### Contour functions

In [0]:
def close_image(image, kernel_size):
    '''Returns the image that is closed with a kernel.'''
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_size, kernel_size))
    closing = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    
    return closing

In [0]:
def extract_longest_contour(image, kernel_size):
    '''Returns the contour with the most points from a given image.'''
    
    # Close image
    closing = close_image(image, kernel_size)
    
    # Convert image to grayscale
    gray = cv2.cvtColor(closing, cv2.COLOR_BGR2GRAY)

    # Apply threshold to turn it into binary image
    ret, thresh = cv2.threshold(gray, 0, 255, 0)

    # Change method for different number of points:
    # CHAIN_APPROX_NONE, CHAIN_APPROX_SIMPLE, CHAIN_APPROX_TC89_L1, CHAIN_APPROX_TC89_KCOS
    _, contours, _ = cv2.findContours(image=thresh, 
                                      mode=cv2.RETR_TREE,
                                      method=cv2.CHAIN_APPROX_NONE)
    
    # Get longest contour
    longest_contour = max(contours, key=len)
    
    # Remove single-dimensional entry from the shape of the array 
    longest_contour = np.squeeze(longest_contour)
    
    return longest_contour

In [0]:
def shorten_contour(contour, points):
    '''Returns a contour with a fixed number of points for a given contour.'''
    
    # TODO: add warning if short_contour has less than stated points
    try:
        step = len(contour) / (len(contour) - points)
    except ZeroDivisionError as e:
        print ('len(contour) = {} smaller, points = {}'.format(
            len(contour), points))
        raise ValueError('len(contour) = {} smaller, points = {}'.format(
            len(contour), points)) from e
        
    short_contour = np.delete(contour, np.arange(0, contour.size, step), axis=0)
    
    return short_contour

In [0]:
def get_contour_from_mask(mask_name, kernel_size, points):
    '''Returns the image, the longest contour, and shortened contour 
       given a mask name and fixed number of points.'''
    
    image = cv2.imread(mask_name)
    contour = extract_longest_contour(image, kernel_size)
    try:
        short_contour = shorten_contour(contour, points)
    except ValueError as e:
        print(mask_name)
        raise ValueError(mask_name) from e
        
    return image, contour, short_contour

## Create contours for all masks

In [0]:
def create_contours_for_all_masks(annotations_folders_csv_id,
                                  annotations_folders_csv_name, 
                                  contours_folder_id,
                                  kernel_size):
    
    # Download csv containing title and id of all video folders
    download_file_from_drive(annotations_folders_csv_id, annotations_folders_csv_name)
    annotations_folders = pd.read_csv(annotations_folders_csv_name, index_col=0)
    
    # Iterate through folders
    for index, row in annotations_folders.iterrows():
        if (index <= 49):
            continue
            
        folder_title = row['title']
        folder_id = row['id']
        print('Folder #{}: {}'.format(index, folder_title))
        
        # Download all images from the folder
        mask_names = download_images_from_folder(folder_id)
        
        # Create folder to save new images with mask and contour
        folder_to_save_id = create_folder_in_drive(contours_folder_id, folder_title)
        
        print('\tCreating contours')
        
        # Iterate through images
        for mask_name in mask_names:

            # If first mask, then extract 256 contour points
            if re.match(r".+mask_00000.png", mask_name):
                try:
                    image, contour, short_contour = get_contour_from_mask(
                        mask_name, kernel_size, 256)
                except ValueError as e:
                    continue
            # If not, then 512 contour points
            else:
                try:
                    image, contour, short_contour = get_contour_from_mask(
                        mask_name, kernel_size, 512)
                except ValueError as e:
                    continue

            # Save image with contour and upload it to drive    
            file_name = 'contour_{}'.format(mask_name)
            save_image_with_contour(image, short_contour, file_name)
            upload_file_to_drive(folder_to_save_id, file_name)

In [0]:
ANNOTATIONS_480P_FOLDER_ID = '1xtR9yVERSGDgk3r605MYk-iSVJDwj7Za'
CONTOURS_480P_FOLDER_ID = '1RK5iZuBRs22UC6kKrK7HCEQQkBXSoPJk'
ANNOTATIONS_480P_FOLDERS_CSV_NAME = 'annotation_folders.csv'
ANNOTATIONS_480P_FOLDERS_CSV_ID = '18XNkwQ3P6ujahicePlVB2CKovH4__Erj'

create_contours_for_all_masks(ANNOTATIONS_480P_FOLDERS_CSV_ID,
                              ANNOTATIONS_480P_FOLDERS_CSV_NAME,
                              CONTOURS_480P_FOLDER_ID,
                              10)